### Recommender system: Day in the city

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.mllib.recommendation import ALS
import requests
import time

In [ ]:
client_id = 'ua-7j9yr_CgcTD_4tQblcA'
api_key = '8dsXYq6nxsBx-XpawRnB6ONyWjpz-vpt0hkk7XGMSh2KfMuoH-KIN-yEow0MBFCuuP6gljF9qgFa58jXB9NuvpY52o6EiSgTX4yncXLPVbIGSaHQi1fBSAWySnJcXHYx'

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    df = pd.DataFrame(response.json()['businesses'])
    return df

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    dfs = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        dfs.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    df = pd.concat(dfs, ignore_index=True)
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

df = all_results(url_params, api_key)
print(len(df))
df.head()

In [14]:
from itertools import combinations, permutations
from bs4 import BeautifulSoup

1. Web Scrape common names

In [93]:
page = f'https://www.ssa.gov/oact/babynames/decades/names2010s.html'

# Make a get request to retrieve the page
html_page = requests.get(page)

# Pass the page content to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser') 
pretty_soup = soup.prettify()

In [94]:
pretty_soup = pretty_soup.strip('').split('\n')

In [96]:
pretty_soup.index('         <th abbr="female number" style="text-align:right; background-color:white; color:black">')

226

In [97]:
pretty_soup = pretty_soup[226:]

In [125]:
a = ''.join(pretty_soup)
a = a.replace('</td>',"")
a = a.replace(' ',"")
a = a.split('<td>')

In [130]:
b = list(filter(lambda x: x.isalpha() , a))

In [134]:
b[:10]

['Noah',
 'Emma',
 'Jacob',
 'Sophia',
 'Mason',
 'Olivia',
 'Liam',
 'Isabella',
 'William',
 'Ava']

In [145]:
page = f'https://www.yelp.com/user_details?userid=3TWGb7-IY3xjfgr2Htsy8w'

# Make a get request to retrieve the page
html_page = requests.get(page)

# Pass the page content to beautiful soup for parsing
soup1 = BeautifulSoup(html_page.content, 'html.parser') 
pretty_soup1 = soup1.prettify()

In [147]:
pretty_soup1.split('\n')

['<!DOCTYPE HTML>',
 '<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->',
 '<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->',
 '<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->',
 '<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->',
 '<!--[if (gt IE 9)|!(IE)]><!-->',
 '<html class="no-js" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml">',
 ' <!--<![endif]-->',
 ' <head>',
 '  <script>',
 '   (function() {',
 '                var main = null;',
 '',
 '                var main=function(){window.onerror=function(k,a,c,i,f){var j=(document.getElementsByTagName("html")[0].getAttribute("webdriver")==="true"||navigator.userAgent==="selenium");var h=f&&(f.name==="ServerSideRenderingError"||f.name===

In [21]:
names = [''.join(p) for p in permutations('abc')]
names

['abc', 'acb', 'bac', 'bca', 'cab', 'cba']

### 2. EDA -  business.json

In [969]:
# reading in file
df = pd.read_json('business.json', lines = True) 
df.describe()

,is_open,latitude,longitude,review_count,stars
count,192609.000000,192609.000000,192609.000000,192609.000000,192609.000000
mean,0.823040,38.541803,-97.594785,33.538962,3.585627
std,0.381635,4.941964,16.697725,110.135224,1.018458
min,0.000000,33.204642,-115.493471,3.000000,1.000000
25%,1.000000,33.637408,-112.274677,4.000000,3.000000
50%,1.000000,36.144815,-111.759323,9.000000,3.500000
75%,1.000000,43.602989,-79.983614,25.000000,4.500000
max,1.000000,51.299943,-72.911982,8348.000000,5.000000


In [970]:
# Dropping Canadian cities

df.postal_code = df.postal_code.apply(lambda x: x if len(x) == 5 else np.nan)

In [971]:
# Dropping Null Attributes column
df = df[~df['postal_code'].isnull()]  
df = df[~df['attributes'].isnull()]    

In [972]:
# Limiting low frequency cities to 500. Below is the list of cities to keep

keep_city = list(pd.DataFrame(df.groupby('city').filter(lambda x : len(x)>500)['city'].value_counts()).reset_index()['index'])
print('Cities to keep:', keep_city)
df.drop(df[~df.city.isin(keep_city)].index, inplace=True)

df.drop(df[df.is_open == 0].index, inplace = True) # dropping permanently closed businesses
df.drop(['is_open'], axis = 1, inplace = True) # dropping because are all open
df.head()


Cities to keep: ['Las Vegas', 'Phoenix', 'Charlotte', 'Scottsdale', 'Pittsburgh', 'Mesa', 'Henderson', 'Tempe', 'Chandler', 'Madison', 'Cleveland', 'Glendale', 'Gilbert', 'Peoria', 'North Las Vegas', 'Champaign', 'Surprise', 'Concord', 'Goodyear', 'Matthews', 'Avondale', 'Huntersville', 'Fort Mill']


,address,attributes,business_id,categories,city,hours,latitude,longitude,name,postal_code,review_count,stars,state
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC
7,"4545 E Tropicana Rd Ste 8, Tropicana","{'RestaurantsPriceRange2': '3', 'GoodForKids':...",gbQN7vr_caG_A1ugSmGhWg,"Hair Salons, Hair Stylists, Barbers, Men's Hai...",Las Vegas,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",36.099872,-115.074574,Supercuts,89121,3,3.5,NV
11,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
16,"4848 E Cactus Rd, Ste 100","{'BusinessAcceptsCreditCards': 'True', 'Busine...",giC3pVVFxCRR89rApqklyw,"Hair Stylists, Beauty & Spas, Hair Salons, Men...",Scottsdale,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-19:0', '...",33.600071,-111.977371,Knot Salon,85254,5,5.0,AZ


In [973]:
# Categories df

cat = pd.read_json('categories.json')

In [974]:
cat.head(2)

,alias,country_blacklist,country_whitelist,parents,title
0,3dprinting,NaN,NaN,[localservices],3D Printing
1,abruzzese,NaN,[IT],[italian],Abruzzese


In [975]:
df.categories = df.categories.apply(lambda x: x.replace(" ",""))

In [976]:
df.categories = df.categories.str.split(",")

In [977]:
df = df.reset_index().drop(['index'],axis = 1)

In [978]:
ls = []
for i in range(df.shape[0]):
    if df.categories[i] == None:
        ls.append(np.nan)
    else:
        x = str(cat.parents[cat.title == df.categories[i][0]])
        ls.append(x[x.index("[")+1:x.index("]")])
df["parents"] = ls

In [979]:
# Restaurants only
df_non_restaurants = df.drop(df[df.parents == 'restaurants'].index)
df = df.drop(df[df.parents != 'restaurants'].index)
df.head()

,address,attributes,business_id,categories,city,hours,latitude,longitude,name,postal_code,review_count,stars,state,parents
9,"Center Core - Food Court, Fl 3, Pittsburgh Int...","{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1RHY4K3BD22FK7Cfftn8Mg,"[Sandwiches, Salad, Restaurants, Burgers, Comf...",Pittsburgh,None,40.496177,-80.246011,Marathon Diner,15231,35,4.0,PA,restaurants
11,6055 E Lake Mead Blvd,"{'BikeParking': 'True', 'BusinessParking': '{'...",tstimHoMcYbkSC4eBA1wEg,"[Mexican, Restaurants, Patisserie/CakeShop, Fo...",Las Vegas,"{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'...",36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156,184,4.5,NV,restaurants
32,436 Market St,"{'OutdoorSeating': 'False', 'HasTV': 'True', '...",v-scZMU6jhnmV955RSzGJw,"[Japanese, SushiBars, Restaurants]",Pittsburgh,"{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",40.441062,-80.002126,No. 1 Sushi Sushi,15222,106,4.5,PA,restaurants
35,"13843 N Tatum Blvd, Ste 15","{'Caters': 'True', 'GoodForKids': 'True', 'Noi...",44YFU284Z3KDEy25QyVoUw,"[Chinese, Restaurants]",Phoenix,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",33.613020,-111.977036,Nee House Chinese Restaurant,85032,269,3.5,AZ,restaurants
69,550 N Hayden Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",nsNONDHbV7Vudqh21uicqw,"[Seafood, Restaurants]",Scottsdale,"{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ...",33.454286,-111.909405,Salt Cellar,85257,526,3.5,AZ,restaurants


In [980]:
# Converting Attributes dictionary in dataframe

attributes_df = df.attributes.apply(pd.Series)
attributes_df = pd.concat([df, attributes_df], axis = 1)

#Dropping non-attribute columns
attributes_df = attributes_df.drop(['address', 'attributes','hours',
                                    'latitude', 'longitude', 'name', 'postal_code'], axis = 1)

In [981]:
#Cleaning Noise Level

ch = {"u'average'": "'average'","u'quiet'": "'quiet'","u'loud'": "'loud'","u'very_loud'": "'very_loud'"}
attributes_df.NoiseLevel = attributes_df.NoiseLevel.replace(to_replace = ch, value=None)

In [982]:
attributes_df.RestaurantsPriceRange2.value_counts()

1       2843
2       2430
3        154
4         34
None       4
Name: RestaurantsPriceRange2, dtype: int64

In [983]:
attributes_df[(attributes_df.review_count > 50) & (attributes_df.GoodForMeal.isnull() == True)].business_id.count()

145

In [984]:
attributes_df = attributes_df[attributes_df.GoodForMeal.isnull() == False]

In [985]:
attributes_df = attributes_df.reset_index().drop(['index'], axis = 1)

In [986]:
goodformeal_df = attributes_df.GoodForMeal.apply(ast.literal_eval)
goodformeal_df = goodformeal_df.apply(pd.Series)
goodformeal_df.head(2)

,dessert,latenight,lunch,dinner,brunch,breakfast
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0


In [987]:
attributes_df = pd.concat([attributes_df, goodformeal_df], axis = 1)

In [988]:
cat_df = attributes_df.categories.apply(lambda x: dict(zip(x, [True]*len(x)))).apply(pd.Series)

In [989]:
attributes_df = pd.concat([attributes_df, cat_df], axis = 1)

In [990]:
attributes_df.shape

(4440, 319)

In [991]:
#Categories to drop : low frequency < 5

cat_drop = []
for col in cat_df.columns:
    if cat_df[col].count() <=5:
        cat_drop.append(col)
len(cat_drop)

137

In [992]:
# Dropping from attributes df low frequency columns

for col in cat_drop:
    attributes_df.drop([col],axis =1, inplace=True)

In [993]:
attributes_df.shape

(4440, 182)

In [994]:
attributes_df = attributes_df[attributes_df.RestaurantsPriceRange2 != 'None']
attributes_df.drop(['categories'],axis = 1)

,business_id,city,review_count,stars,state,parents,RestaurantsTakeOut,BusinessParking,Ambience,RestaurantsDelivery,...,Kebab,HealthMarkets,HotPot,MusicVenues,Kosher,Live/RawFood,Afghan,African,Persian/Iranian,Smokehouse
0,1RHY4K3BD22FK7Cfftn8Mg,Pittsburgh,35,4.0,PA,restaurants,True,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tstimHoMcYbkSC4eBA1wEg,Las Vegas,184,4.5,NV,restaurants,True,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'classy...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,v-scZMU6jhnmV955RSzGJw,Pittsburgh,106,4.5,PA,restaurants,True,"{'garage': False, 'street': True, 'validated':...","{'touristy': False, 'hipster': False, 'romanti...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,44YFU284Z3KDEy25QyVoUw,Phoenix,269,3.5,AZ,restaurants,True,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nsNONDHbV7Vudqh21uicqw,Scottsdale,526,3.5,AZ,restaurants,False,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6fPQJq4f_yiq1NHn0fd11Q,Las Vegas,535,3.5,NV,restaurants,True,"{'garage': True, 'street': False, 'validated':...","{'romantic': False, 'intimate': False, 'touris...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,k-dDZvTeLysoJvjHI-qr9g,Las Vegas,287,3.0,NV,restaurants,False,"{'garage': True, 'street': False, 'validated':...","{'romantic': False, 'intimate': False, 'touris...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,F6c3D1o9Z4Tl6cDorb3WgA,Pittsburgh,75,3.5,PA,restaurants,True,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'classy...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,hXzoNgpkC86K_Jfg_zMHvA,Glendale,96,4.5,AZ,restaurants,True,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AMN3Sgns9tB2x-b3eWk0qQ,Scottsdale,457,4.0,AZ,restaurants,True,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [995]:
for col in attributes_df.columns:
    print(attributes_df[col].value_counts())

0wW9PasC8pw8SY7rlY3ZKw    1
IdneSH2aKUz9c8K5OHYg7w    1
F2L2f-ITjnHlqKRFQVFsVg    1
TM3UY0fCq0CQbuskoq0OWQ    1
0FBnMCA0-V-FJkAbwU1wqg    1
RQdKieW9hqg898d_S2WwRA    1
FSnqO-YWUdRb0Xc95nXbvw    1
OQH2cl8SPWp9HR5S_CXFgA    1
YavRvjbLsvh4DYDJPAfJIA    1
0_aeYE2-VbsZts_UpILgDw    1
ti1bb30WwnkDLuVvpTM6fQ    1
9ULcHyUTN1O16Vr8KUMQew    1
HMSDOjt_KCyknzjQ9aI5Jw    1
6-VJLO3Phc8uenJaLH54Og    1
nFVqjh-JCOuV2Uci1ZfJ_Q    1
kE8hSuvO6eKtbRA1k541Jg    1
cq6SQkWKVT3TH-6YzGFn4g    1
8F9C5vhDlYphBWELUgLkvQ    1
uGCct-R33OXz53igyKjejQ    1
qLO9ES3-mKJrodFO1mzgOg    1
pyqnGlIfP9Zw8LLxBjyAiA    1
65IpGk7kDCsAbphNtQJJRw    1
L1KkmY4vTgCrS1J5aUFMtA    1
Jg0mxuXZbuCKI91TQtKwDA    1
6vNMmkttsHkW1THWiP50xg    1
vdZOiZH9sTseyll44waMuA    1
9Xsjz292QEFVff_p03qsZQ    1
G-5kEa6E6PD5fkBRuA7k9Q    1
PWjaIjXAXXfs_I4TnAfvtQ    1
6llKs7K_tn8ChXcIM-oTvg    1
                         ..
PJVVjbmwNbxqfvFi3f3hVA    1
o01ztaNP08Dq8DJImuovcw    1
PYoGpgAUDgT6OMxD_fe79g    1
KUOa1acSFn6DkO9jp-sn6A    1
_laY6Ba-VWXCR6xg7LVt

Name: StreetVendors, dtype: int64
True    20
Name: Filipino, dtype: int64
True    17
Name: Spanish, dtype: int64
True    18
Name: TapasBars, dtype: int64
True    9
Name: DiveBars, dtype: int64
True    17
Name: Tapas/SmallPlates, dtype: int64
True    9
Name: Teppanyaki, dtype: int64
True    12
Name: Mongolian, dtype: int64
True    10
Name: NewMexicanCuisine, dtype: int64
True    31
Name: Wraps, dtype: int64
True    9
Name: Breweries, dtype: int64
True    9
Name: Irish, dtype: int64
True    7
Name: Brasseries, dtype: int64
True    7
Name: Cafeteria, dtype: int64
True    27
Name: SoulFood, dtype: int64
True    18
Name: BeerBar, dtype: int64
True    24
Name: Lounges, dtype: int64
True    19
Name: Poke, dtype: int64
True    6
Name: Belgian, dtype: int64
True    7
Name: Lebanese, dtype: int64
True    23
Name: Szechuan, dtype: int64
True    7
Name: ImportedFood, dtype: int64
True    13
Name: DimSum, dtype: int64
True    6
Name: Hotels&Travel, dtype: int64
True    10
Name: Brazilian, dtype: in

In [996]:
clean_df = attributes_df.drop(['categories','city','state','parents','RestaurantsTakeOut',
                   'RestaurantsDelivery','DriveThru','GoodForMeal','WiFi',
                   'Caters','RestaurantsTableService','BYOBCorkage','Corkage',
                   'BYOB','CoatCheck','Smoking','GoodForDancing','DogsAllowed',
                   'BusinessAcceptsBitcoin','ByAppointmentOnly','AgesAllowed',
                   'Open24Hours','DietaryRestrictions','RestaurantsCounterService',
                   'AcceptsInsurance','Music','BestNights', 'HappyHour'],axis = 1)

In [997]:
# Number of Ambience null values to drop
sum(clean_df.Ambience.isnull())

116

In [998]:
# Unpacking Ambience dictionary & dropping null values

clean_df = clean_df[clean_df.Ambience.isnull() == False]
ambience_df = clean_df.Ambience.apply(ast.literal_eval)
ambience_df = ambience_df.apply(pd.Series)
clean_df = pd.concat([clean_df, ambience_df], axis = 1)
clean_df.drop(['Ambience'],axis = 1,inplace = True)
clean_df.head(2)

,business_id,review_count,stars,BusinessParking,RestaurantsReservations,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsGoodForGroups,GoodForKids,HasTV,...,Smokehouse,touristy,hipster,romantic,divey,intimate,trendy,upscale,classy,casual
0,1RHY4K3BD22FK7Cfftn8Mg,35,4.0,"{'garage': False, 'street': False, 'validated'...",False,True,1,True,True,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tstimHoMcYbkSC4eBA1wEg,184,4.5,"{'garage': False, 'street': False, 'validated'...",False,True,1,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [999]:
clean_df[clean_df.BusinessParking.isnull() == True]

,business_id,review_count,stars,BusinessParking,RestaurantsReservations,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsGoodForGroups,GoodForKids,HasTV,...,Smokehouse,touristy,hipster,romantic,divey,intimate,trendy,upscale,classy,casual
30,iE71iwcSljg3xm2GB2Y9aA,497,4.0,NaN,True,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
50,lqPuSiWypNCACEMhArSZ2A,22,2.5,NaN,False,True,1,True,True,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,gPZj-XXVwFHgd8zjJPzVCw,12,3.5,NaN,False,True,1,True,True,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,QzvLnOqwH6BIY_jCOvzuQQ,30,2.5,NaN,False,True,2,False,False,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,xVWLSFGNr4w4DYvYiafMPw,10,4.0,NaN,False,True,1,True,True,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174,9q8phddM1YWNKZVepKi6iQ,10,3.0,NaN,False,True,1,True,True,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189,8CjI8nebSDB5aINJc3dLwA,13,4.0,NaN,True,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223,Kzo1z1hW6224lNd3al2SbQ,29,3.5,NaN,False,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
236,oNCF7oHyKYlGLu_kHV2MSg,9,2.0,NaN,True,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,MYB1ZMspBk1Xc_awp_PtSw,79,3.5,NaN,False,True,2,True,False,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [1000]:
clean_df[clean_df.BusinessParking.isnull() == True]

,business_id,review_count,stars,BusinessParking,RestaurantsReservations,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsGoodForGroups,GoodForKids,HasTV,...,Smokehouse,touristy,hipster,romantic,divey,intimate,trendy,upscale,classy,casual
30,iE71iwcSljg3xm2GB2Y9aA,497,4.0,NaN,True,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
50,lqPuSiWypNCACEMhArSZ2A,22,2.5,NaN,False,True,1,True,True,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,gPZj-XXVwFHgd8zjJPzVCw,12,3.5,NaN,False,True,1,True,True,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,QzvLnOqwH6BIY_jCOvzuQQ,30,2.5,NaN,False,True,2,False,False,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,xVWLSFGNr4w4DYvYiafMPw,10,4.0,NaN,False,True,1,True,True,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174,9q8phddM1YWNKZVepKi6iQ,10,3.0,NaN,False,True,1,True,True,False,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189,8CjI8nebSDB5aINJc3dLwA,13,4.0,NaN,True,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223,Kzo1z1hW6224lNd3al2SbQ,29,3.5,NaN,False,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
236,oNCF7oHyKYlGLu_kHV2MSg,9,2.0,NaN,True,True,2,True,True,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,MYB1ZMspBk1Xc_awp_PtSw,79,3.5,NaN,False,True,2,True,False,True,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [1001]:
# Unpacking Parking column - replacing null values by no street parking

parking_df = clean_df.BusinessParking.replace(to_replace = np.nan, value = "{'street':False}")
parking_df = parking_df.apply(ast.literal_eval)
parking_df = parking_df.apply(pd.Series)

In [1002]:
# Assigning Boolean value to parking: True if garage, validated, lot, valet parking available; False otherwise.

parking_df['parking'] = parking_df['garage']|parking_df['validated']|parking_df['lot']|parking_df['valet']
parking_df.drop(['garage','street','validated','lot','valet'], axis = 1, inplace=True)
clean_df = pd.concat([clean_df, parking_df], axis = 1)
clean_df.drop(['BusinessParking'],axis = 1,inplace = True)

In [1003]:
#Cleaning Restaurants Attire

attire = {"u'casual'": "'casual'","u'dressy'": "'dressy'","u'formal'":"'dressy'","'formal'":"'dressy'"}
clean_df.RestaurantsAttire = clean_df.RestaurantsAttire.replace(to_replace = attire, value=None)

In [1004]:
#Cleaning Alcohol

alc = {"u'none'": "False","'none'": "False","None": "False","u'full_bar'": "True","'full_bar'": "True",
      "u'beer_and_wine'": "True","'beer_and_wine'":"True"}
clean_df.Alcohol = clean_df.Alcohol.replace(to_replace = alc, value=None)

In [1005]:
clean_df.WheelchairAccessible.value_counts()

True     954
False     52
None       4
Name: WheelchairAccessible, dtype: int64

In [1006]:
clean_df.head(3)

,business_id,review_count,stars,RestaurantsReservations,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsGoodForGroups,GoodForKids,HasTV,OutdoorSeating,...,touristy,hipster,romantic,divey,intimate,trendy,upscale,classy,casual,parking
0,1RHY4K3BD22FK7Cfftn8Mg,35,4.0,False,True,1,True,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1,tstimHoMcYbkSC4eBA1wEg,184,4.5,False,True,1,True,True,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
2,v-scZMU6jhnmV955RSzGJw,106,4.5,False,True,2,False,True,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,False


In [1007]:
# Filling NA values

clean_df.RestaurantsPriceRange2.fillna(1, inplace = True)
clean_df.RestaurantsPriceRange2.isna().sum()

0

In [1009]:
clean_df.RestaurantsAttire.value_counts()

'casual'    4121
'dressy'      91
Name: RestaurantsAttire, dtype: int64

In [1029]:
# Wheelchair access

wch = {False:0, True:1,'False':0, 'True':1'None':0}
clean_df.WheelchairAccessible = clean_df.WheelchairAccessible.replace(to_replace = wch, value=None)
clean_df.GoodForKids = clean_df.GoodForKids.replace(to_replace = wch, value=None)
clean_df.HasTV = clean_df.HasTV.replace(to_replace = wch, value=None)
clean_df.OutdoorSeating = clean_df.OutdoorSeating.replace(to_replace = wch, value=None)
clean_df.BikeParking = clean_df.BikeParking.replace(to_replace = wch, value=None)
clean_df.Alcohol = clean_df.Alcohol.replace(to_replace = wch, value=None)
clean_df.BusinessAcceptsCreditCards = clean_df.BusinessAcceptsCreditCards.replace(to_replace = wch, value=None)
clean_df.RestaurantsReservations = clean_df.RestaurantsReservations.replace(to_replace = wch, value=None)
clean_df.RestaurantsGoodForGroups = clean_df.RestaurantsGoodForGroups.replace(to_replace = wch, value=None)

clean_df.RestaurantsAttire.fillna("'casual'", inplace = True)  

clean_df.NoiseLevel = clean_df.NoiseLevel.replace('None', value="'average'")
clean_df.NoiseLevel.fillna("'average'", inplace = True)  

clean_df.WheelchairAccessible.fillna(2, inplace = True)  # 2 means unknown
clean_df.fillna(0, inplace=True)
clean_df = clean_df.replace(to_replace = wch, value=None)

In [1020]:
# change type
clean_df.RestaurantsPriceRange2 = clean_df.RestaurantsPriceRange2.astype(int)

### Part 2: Modeling

In [1040]:
#!conda install -y -c conda-forge scikit-surprise
from surprise import SVD
from surprise.model_selection import cross_validate

In [1042]:
X = clean_df.drop(['business_id'], axis = 1)

#### Part 2 A: Clustering

In [ ]:
from sklearn.cluster import KMeans # import from sklearn
k_means = KMeans(n_clusters=3) # Must set number of clusters at initialization time!
from sklearn.metrics import calinski_harabaz_score


print(calinski_harabaz_score(some_df, cluster_assignments))